# Regresion_Lineal (cont.)
## Ejemplo de Análisis de Regresión de una sola ecuación

In [ ]:
# !pip install LIBRERIA

In [ ]:
# Dependencias
import pyreadstat #librería para leer formato ".dta"
import pandas as pd #librería para manipulación de datos
import numpy as np #Librería para operaciones matemáticas
import matplotlib #librería para graficar
from matplotlib import pyplot as plt #librería para graficar
import statsmodels.api as sm #librería para análisis estadístico
from IPython.display import Image # Para imagénes
from statsmodels.formula.api import ols # Para pruebas de hipotesis

#
import warnings
warnings.filterwarnings('ignore')

### Metadatos: La base de datos contiene 145 observaciones  de las siguientes variables:
**Datos de Demanda de Gasolina:**
* **totcost:** costs in 1970, MM USD

* **output:** output billion KwH
* **plabor:** price of labor
* **pfuel:** price of fuel
* **pkap:** price of capital

In [ ]:
# Ruta al archivo:
dtafile = 'nerlove63.dta'
# La función read_dat devuelve dos objetos: el dataframe y los metadatos
dataframe, meta = pyreadstat.read_dta(dtafile)

In [ ]:
dataframe.head(5)

## Aplicación del Método de MCO: Retornos a Escala en la Industria Eléctrica
Considerando lo anterior plateamos la expresión:
$$ln(C_i) = \beta_1 +  \beta_2 ln(Q_i) + \beta_3 ln(p_{i1}) + \beta_4 ln(p_{i2}) + \beta_5 ln(p_{i3}) + \varepsilon_i$$

Donde:
$$\beta_1 = \mu$$

$$\beta_2 = \frac{1}{r}$$

$$\beta_3 = \frac{\alpha_1}{r}$$

$$\beta_4 = \frac{\alpha_2}{r}$$

$$\beta_5 = \frac{\alpha_3}{r}$$

De esta forma podemos decir que $y_i = ln(C_i)$ y que:
$$\mathbf{X}'_i = [1, ln(Q_i), ln(p_{i1}), ln(p_{i2}), ln(p_{i3})]$$


### Transformación de datos:

In [ ]:
#Creamos la variable Ltotcost aplicando logaritmo a totcost y guardamos el resultado en la nueva columna Ltotcost
dataframe['Ltotcost'] = np.log(dataframe['totcost'])
dataframe['Loutput'] = np.log(dataframe['output'])
dataframe['Lplabor'] = np.log(dataframe['plabor'])
dataframe['Lpfuel'] = np.log(dataframe['pfuel'])
dataframe['Lpkap'] = np.log(dataframe['pkap'])
dataframe['avgcost'] = dataframe["totcost"]/dataframe["output"]
dataframe['One'] = 1

In [ ]:
#
dataframe.head()

## Regresión base: 

In [ ]:
# Definición de variables:
Y = dataframe["Ltotcost"]
X = dataframe[["One","Loutput", "Lplabor", "Lpfuel", "Lpkap"]]
#
X

In [ ]:
# Estimación:
est = sm.OLS(Y,X)
est2 = est.fit()
print(est2.summary())

Estimadores:
$$\hat{\boldsymbol \beta} = (\mathbf{X'X})^{-1}\mathbf{X'Y}$$

In [ ]:
#
est2.params

La especificación más común de la prueba de hipótesis en el análisis de regresión es:
$$H_0: \beta_k = 0$$
$$H_a: \beta_k \neq 0$$

Lo que en términos de una prueba $t$ es la siguiente:
$$t = \frac{\hat{\beta}_k - 0}{\sqrt{\hat{\sigma}^2 (\mathbf{X}'\mathbf{X})^{-1}_{kk}}} \sim t_{n - k}$$

In [ ]:
#
est2.tvalues

La hipotésis nula de una prueba global se puede escribir como:
$$H_0: \beta_1 = \beta_2 = \ldots = \beta_K = 0$$
$$H_a: No H_0$$

Esta prueba se le conoce como prueba global, ya que prueba si en conjunto todas las variables independientes tienen un efecto nulo en $\mathbf{Y}$. Notemos que la prueba implica que la estadística $F$ de prueba será:
$$F = \frac{(\mathbf{R} \boldsymbol{\hat{\beta}} - \mathbf{r})'[\mathbf{R} \hat{\sigma}^2 (\mathbf{X}'\mathbf{X})^{-1}\mathbf{R}']^{-1}(\mathbf{R} \boldsymbol{\hat{\beta}} - \mathbf{r})}{J} \sim F_{J, n - K}$$    

La matriz $\mathbf{R}$ y vector $\mathbf{r}$:
$$\mathbf{R} = \left[ 
            \begin{array}{c c c c}
                1 & 0 & \ldots & 0 \\ 0 & 1 & \ldots & 0 \\ 0 & 0 & \ldots & 0 \\ \vdots & \vdots & \cdots & \vdots \\ 0 & 0 & \ldots & 1 \\
            \end{array}
            \right]$$

$$\mathbf{r} = \left[ 
            \begin{array}{c}
                0 \\ 0 \\ 0 \\ \vdots \\ 0 \\
            \end{array} 
            \right]$$


In [ ]:
#
est2.fvalue

Alternativamente:

In [ ]:
#
R = np.array(([0,0,0,0,0], [0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]))
R

In [ ]:
#
print(est2.f_test(R))

En el caso que nos ocupa queremos probar si la suma de los coeficientes asociados los factores productivos es 1, es decir, la función de costos exibe rendimientos constantes a escala:
$$H_0: \beta_3 + \beta_4 + \beta_5 = 1$$
$$H_1: \beta_3 + \beta_4 + \beta_5 \neq 1$$

Existen dos alternativas para probar la hipótesis. Partamos de que:
La matriz $\mathbf{R}$ y vector $\mathbf{r}$:
$$\mathbf{R} = \left[ 
            \begin{array}{c c c c c}
                0 & 0 & 1 & 1 & 1 
            \end{array}
            \right]$$

$$\mathbf{r} = 1$$


In [ ]:
#
formula = 'Ltotcost ~  One + Loutput + Lplabor + Lpfuel + Lpkap'
results = ols(formula, dataframe).fit()
# Se pueden incluir tantas restricciones queramos, agregando una coma (,) segido de la restricción
hypotheses = 'Lplabor + Lpfuel + Lpkap = 1'

$$H_0 = \beta_3 + \beta_4 + \beta_5 = 1$$
$$H_0 = \beta_3 + \beta_4 + \beta_5 - 1 = 0$$

Lo que en términos de una prueba $t$ es la siguiente:
$$t = \frac{\hat{\beta}_k - 1}{\sqrt{\hat{\sigma}^2 R (\mathbf{X}'\mathbf{X})^{-1} R'}} \sim t_{n - k}$$

In [ ]:
# 1:
t_test = results.t_test(hypotheses)
print(t_test)

$$F = \frac{(\mathbf{R} \boldsymbol{\hat{\beta}} - \mathbf{1})'[\mathbf{R} \hat{\sigma}^2 (\mathbf{X}'\mathbf{X})^{-1}\mathbf{R}']^{-1}(\mathbf{R} \boldsymbol{\hat{\beta}} - \mathbf{1})}{1} \sim F_{1, n - K}$$    


In [ ]:
# 2:
f_test = results.f_test(hypotheses)
print(f_test)

$$H_0 = \beta_3 + \beta_4 + \beta_5 = 1, \beta_2 = 0$$
$$H_1 = No H_0$$

In [ ]:
# Otro ejemplo: 
hypotheses_2 = 'Lplabor + Lpfuel + Lpkap = 1, Loutput = 0'
f_test = results.f_test(hypotheses_2)
print(f_test)

In [ ]:
print(results)

Cómo se ve la curva de costos medios estimados:

In [ ]:
# Rcuperamos de la regresión el valor del Log del costo estimado
LY_pred = est2.predict(X)
# Anti-log:
Y = np.exp(LY_pred)
# Colocamos en el Data Frame:
dataframe['totcost_e'] = Y
dataframe['avgcost_e'] = dataframe["totcost_e"]/dataframe["output"]
dataframe.head()

In [ ]:
# graficamos resultados:
plt.scatter(dataframe.output, dataframe.avgcost, s = 15, color ="red")
plt.scatter(dataframe.output, dataframe.avgcost_e, s = 15, color ="blue")
plt.title("Gráfico de dispersión Output vs Avg cost / Avg cost estimado")
#
plt.show()

# Ejercicio:

Estimar:
$$ln(C_i) = \beta_1 +  \beta_2 ln(Q_i) + \beta_2 (ln(Q_i))^2 + \beta_3 ln(p_{i1}) + \beta_4 ln(p_{i2}) + \beta_5 ln(p_{i3}) + \varepsilon_i$$

Probar si:
$$H_0: \beta_3 + \beta_4 + \beta_5 = 1$$
$$H_1: \beta_3 + \beta_4 + \beta_5 \neq 1$$


In [ ]:
dataframe.head()

In [ ]:
dataframe["Loutput_2"] = dataframe["Loutput"]**2

In [ ]:
dataframe.head()

In [ ]:
# Definición de variables:
Y = dataframe["Ltotcost"]
X = dataframe[["One", ]]
#
X

In [ ]:
# Estimación:
est = sm.OLS(Y,X)
est2 = est.fit()
print(est2.summary())

In [ ]:
formula = 'Ltotcost ~  One + '
results = ols(formula, dataframe).fit()

# Se pueden incluir tantas restricciones queramos, agregando una coma (,) segido de la restricción
hypotheses = ' +  +  = 1'
t_test = results.t_test(hypotheses)
print(t_test)